# With Bert encoding and SNN model

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'./cleaned_data/full_file.csv')
df.head()

df = df[['reviewText','class']]
df.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_1640\1707058494.py:4: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'./cleaned_data/full_file.csv')


,reviewText,class
0,i bought these because of the rave reviews and...,0
1,its a cute case.. but its really hard to get o...,0
2,these had excellent sound quality and felt dur...,0
3,the headphones i got were either defective or ...,0
4,it is a good product with a good sound quality...,0


In [2]:
# # Function to count words in a column
# def count_words(text):
#     return len(str(text).split())

# # Apply the function to count words in a column
# word_count = df['reviewText'].apply(count_words)


In [3]:
# max(word_count)

In [4]:
# import seaborn as sns
# sns.boxplot(word_count)

In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import pandas as pd

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [7]:
df2 = df.sample(2000000,random_state=42)
df2.rename(columns={'reviewText':'text','class':'label'},inplace=True)
df2 = df2[df2['text'].str.len()<=512]

In [8]:
df2['label'].value_counts(), df2.shape

(1    836189
 0    782331
 Name: label, dtype: int64,
 (1618520, 2))

In [9]:
df.shape

(9457245, 2)

### Encoding using Bert

In [10]:
# If it's already in 'text' and 'label', you can skip this step
texts = df2['text'].tolist()
labels = df2['label'].tolist()

# Split the dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(set(labels)))

# Tokenize the texts
max_len = 512  # Set maximum sequence length
train_encodings = tokenizer(train_texts, truncation=True, padding='max_length', max_length=max_len)
test_encodings = tokenizer(test_texts, truncation=True, padding='max_length', max_length=max_len)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# [len(i) for i in test_encodings['input_ids']]

### converting to pytorch tensors

In [11]:
# Convert to PyTorch tensors
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])
test_input_ids = torch.tensor(test_encodings['input_ids'])
test_attention_mask = torch.tensor(test_encodings['attention_mask'])
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Create PyTorch datasets
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)

# Set training parameters
batch_size = 8
epochs = 10
learning_rate = 1e-5


### Model definition

In [42]:

# # Define the Sequential NN model
# class SequentialModel(nn.Module):
#     def __init__(self, input_size):
#         super(SequentialModel, self).__init__()        
#         self.seq_model = nn.Sequential(
#             nn.Linear(input_size, 512),
#             nn.Dropout(p=0.3),
#             nn.Linear(512, 258),
#             nn.Dropout(p=0.1),
#             nn.Linear(258, 124),
#             nn.Dropout(p=0.1),
#             nn.Linear(124, 1),
#             nn.Sigmoid()
#         )

#     def forward(self, inputs):

#         return self.seq_model(inputs)

# # Instantiate the model
# model = SequentialModel(input_size=max_len)

# # Move model to GPU if available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# # Define loss function and optimizer
# criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Create DataLoaders
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [12]:

class SequentialModel(nn.Module):
    def __init__(self, input_dim):
        super(SequentialModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)  # Output dimension is 1 for binary classification

    def forward(self, x):
        out = torch.sigmoid(self.linear(x))  # Sigmoid activation for logistic regression
        return out

# Instantiate the model
model = SequentialModel(input_dim=max_len)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [13]:
device

device(type='cuda')

In [14]:
print(model)

SequentialModel(
  (linear): Linear(in_features=512, out_features=1, bias=True)
)


In [15]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

###  Model training

In [16]:
from tqdm import tqdm
# Training loop
for epoch in tqdm(range(epochs)):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for input_ids, attention_mask, labels in train_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        y_pred = model(input_ids.float())
        
        loss = criterion(y_pred.squeeze(), labels.float())
        acc = binary_acc(y_pred.squeeze(), labels.float())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {epoch+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

 10%|████████▏                                                                         | 1/10 [05:23<48:33, 323.72s/it]

Epoch 000: | Loss: 48.26784 | Acc: 51.607


 20%|████████████████▍                                                                 | 2/10 [09:21<36:23, 272.92s/it]

Epoch 001: | Loss: 48.30465 | Acc: 51.686


 30%|████████████████████████▌                                                         | 3/10 [13:09<29:28, 252.63s/it]

Epoch 002: | Loss: 48.30603 | Acc: 51.685


 40%|████████████████████████████████▊                                                 | 4/10 [16:57<24:18, 243.03s/it]

Epoch 003: | Loss: 48.30652 | Acc: 51.684


 40%|████████████████████████████████▊                                                 | 4/10 [17:27<26:11, 261.95s/it]


KeyboardInterrupt: 

### Evaluation

In [18]:
# Evaluation
model.eval()
all_preds = []
true_labels = []
with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids.float())
        preds = (outputs > 0.5).squeeze().int()
        all_preds.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(true_labels, all_preds)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5539350764896325


In [19]:
import pickle
# Save the model to a file using pickle
with open('./SNN.pkl', 'wb') as file:
    pickle.dump(model, file)